<a href="https://colab.research.google.com/github/khodid/2020_SAI_MONING2/blob/master/lab08-1_code_Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Layer Perceptron

이번에는 여러 레이어로 이루어진 모델을 학습시켜, XOR 문제에 대한 제대로 된 결과를 내놓는 걸 관찰하겠다.

## 패키지 import

In [0]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

## Back Propagation 실습

In [0]:
# Training Data
X = torch.FloatTensor([[0, 0],[0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# layers
w1 = torch.Tensor(2, 2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2, 1).to(device)
b2 = torch.Tensor(1).to(device)

레이어는 이런 식으로 구성되어 있다.

$$
(X_{train} \cdot w_1 + b_1)\cdot w_2 + b_2
$$

이를 좀더 풀어서 쓰면,
여기서 첫번째 레이어는

$$
\begin{bmatrix}
x_1 & x_2
\end{bmatrix}
\cdot
\begin{bmatrix}
w_{a1} & w_{b1} \\ w_{a2} & w_{b2}
\end{bmatrix}
+
\begin{bmatrix}
b_a & b_b
\end{bmatrix}
$$
$$
=
\begin{bmatrix} 
x_1 \cdot w_{a1} + x_2 \cdot w_{a2} + b_a  
& 
x_1 \cdot w_{b1} + x_2 \cdot w_{b2} + b_b
\end{bmatrix}
$$

라는 결과에서 정답이 0인지 1인지를 판별하고,
두 번째 레이어에서는

$$
\begin{bmatrix} 
x_{next1}  
& 
x_{next2}  
\end{bmatrix}
\cdot
\begin{bmatrix} 
 w_{2a}
 \\
 w_{2b}
\end{bmatrix}
+
\begin{bmatrix} 
b_{2a}  
& 
b_{2b}
\end{bmatrix}
$$

라는 연산을 해서 결과를 내놓는다.


In [0]:
# 시그모이드도 PyTorch 라이브버리에서 쓰지 않고 직접 짜보겠다.

def sigmoid(x):
  return 1.0 / (1.0 + torch.exp(-x))

# 시그모이드 함수 미분꼴
def sigmoid_prime(x):
  return sigmoid(x) * (1-sigmoid(x))

In [11]:

for step in range(10001):
  
  # forward

  # 위에서 언급한 수식을 l1, l2로 구현.
  l1 = torch.add(torch.matmul(X, w1), b1)
  a1 = sigmoid(l1)
  l2 = torch.add(torch.matmul(a1, w2), b2)
  Y_pred = sigmoid(l2)

  # Cost는 Binary Cross Entropy
  cost = -torch.mean(Y* torch.log(Y_pred) + (1-Y)*torch.log(1-Y_pred))

  #Back propagation (chain rule)

  # BCE를 미분한 식.
  d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7) # 1e-7은 분모가 0 되는 걸 막는 상수

  # Layer 2
  d_l2 = d_Y_pred * sigmoid_prime(l2)
  d_b2 = d_l2
  d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_b2) 

  # Layer 1
  d_a1 = torch.matmul(d_b2, torch.transpose(w2, 0, 1))
  d_l1 = d_a1 * sigmoid_prime(l1)
  d_b1 = d_l1
  d_w1 = torch.matmul(torch.transpose(X, 0, 1), d_b1)


  # Weight update
  ## Gradient Descent
  w1 = w1 - learning_rate * d_w1
  b1 = b1 - learning_rate * torch.mean(d_b1, 0)
  w2 = w2 - learning_rate * d_w2
  b2 = b2 - learning_rate * torch.mean(d_b2, 0)

  if (step % 100 == 0 and step < 2000) or (step % 400 == 0):
    print(step, cost.item())
  

0 0.6931471824645996
100 0.6931471824645996
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471228599548
700 0.6931452751159668
800 0.6930633187294006
900 0.6412245035171509
1000 0.2531474232673645
1100 0.03129035234451294
1200 0.016702039167284966
1300 0.01122652180492878
1400 0.008407686837017536
1500 0.006701599806547165
1600 0.0055622230283916
1700 0.004749101586639881
1800 0.0041405013762414455
1900 0.0036683687940239906
2000 0.003291659988462925
2400 0.002328727161511779
2800 0.0017987260362133384
3200 0.001463932334445417
3600 0.001233537564985454
4000 0.0010654086945578456
4400 0.0009373538196086884
4800 0.0008366158581338823
5200 0.0007553243776783347
5600 0.0006883729947730899
6000 0.0006322836852632463
6400 0.0005845638224855065
6800 0.0005435418570414186
7200 0.0005078301182948053
7600 0.0004765183839481324
8000 0.00044881596113555133
8400 0.0004242006689310074
8800 0.00040207590791396797
9200 0.0003821431891992688
9600 

cost가 점차 줄어드는 걸 관찰할 수 있다!

transpose는 전치행렬을 생각하면 된다.
뒤에 들어가는 인자는 바꿔줄 차원 방향을 정해주는 것.

참고로 위의 
d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred))
의 그래프를 그려보면,
Y_pred = 0과 Y_pred = 1을 점근선으로 가지고 Y_pred가 Y값과 일치하는 지점에서 0이 되는 그래프가 나온다.

![https://ibb.co/1KmCWGP](https://i.ibb.co/QnKWwvT/dl2.png)

대충 이런 느낌임...


## 실전 - nn 패키지 이용

아까까진 정말 원론적으로 이론대로 구현시키는 방법이었고,
매 번 모델을 만들 때마다 저렇게 길고 수학적인 공식을 만들 수는 없는 법이니 패키지를 사용하는 방법을 알아보도록 하자.

In [13]:
# Training Data
X = torch.FloatTensor([[0, 0],[0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)


# nn Layers
linear1 = torch.nn.Linear(2, 2, bias = True)
linear2 = torch.nn.Linear(2, 1, bias = True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

# cost function과 optimizer 설정
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

# 학습
for step in range(10001):
  optimizer.zero_grad()
  hypothesis = model(X)

  cost = criterion(hypothesis, Y)
  cost.backward()
  optimizer.step()

  if (step % 100 == 0 and step < 1000) or (step % 500 ==0):
    print(step, cost.item())

0 0.7126196026802063
100 0.6933621168136597
200 0.6932636499404907
300 0.6932241916656494
400 0.693203866481781
500 0.6931912899017334
600 0.6931823492050171
700 0.6931754350662231
800 0.6931697130203247
900 0.6931648254394531
1000 0.6931604743003845
1500 0.6931399703025818
2000 0.6930841207504272
2500 0.6895962357521057
3000 0.04585985466837883
3500 0.013895433396100998
4000 0.00805296003818512
4500 0.005646144971251488
5000 0.004339577630162239
5500 0.003520903643220663
6000 0.002960511948913336
6500 0.0025531407445669174
7000 0.0022437721490859985
7500 0.0020009279251098633
8000 0.0018052862724289298
8500 0.0016442961059510708
9000 0.0015095683047547936
9500 0.0013951826840639114
10000 0.0012968203518539667


1000 이전엔 100 step마다 출력하고, 그 이후론 500마다 출력하게 했다.

10000번 이후로도 계속 loss가 감소하는 걸 확인할 수 있다.

## nn 패키지로 다층 레이어 쌓아보기

방금 전에는 레이어 두 개를 이용했는데, 이번엔 레이어를 좀더 많이 쌓아보는 코드를 살펴보자.



In [14]:
# Training Data
X = torch.FloatTensor([[0, 0],[0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)


# nn Layers : build 4 layers
linear1 = torch.nn.Linear(2, 10, bias = True)
linear2 = torch.nn.Linear(10, 10, bias = True)
linear3 = torch.nn.Linear(10, 10, bias = True)
linear4 = torch.nn.Linear(10, 1, bias = True)
sigmoid = torch.nn.Sigmoid()

# 모델 설정
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if (step % 100 == 0 and step < 2000) or (step % 500 ==0):
        print(step, cost.item())



0 0.6937791109085083
100 0.6931153535842896
200 0.6931127309799194
300 0.693109929561615
400 0.6931067705154419
500 0.6931031942367554
600 0.6930992007255554
700 0.6930948495864868
800 0.6930898427963257
900 0.693084180355072
1000 0.693077564239502
1100 0.6930699348449707
1200 0.693061113357544
1300 0.6930506229400635
1400 0.6930381655693054
1500 0.6930232644081116
1600 0.6930047869682312
1700 0.6929820775985718
1800 0.6929532289505005
1900 0.6929158568382263
2000 0.6928662657737732
2500 0.691940188407898
3000 0.5511929988861084
3500 0.002833011094480753
4000 0.0011025737039744854
4500 0.0006645479006692767
5000 0.00046999045298434794
5500 0.00036121075390838087
6000 0.0002922551066149026
6500 0.00024469252093695104
7000 0.0002100691490340978
7500 0.00018377824744675308
8000 0.00016316630353685468
8500 0.00014654890401288867
9000 0.00013292730727698654
9500 0.00012157116725575179
10000 0.00011192896636202931


같은 step 수인데도 cost가 훨씬 더 많이 줄어든다는 걸 확인할 수 있다.